# Experiment 0

## To Do

* Function that appends objective values to boards
* Fuction that recreates game in real time as movie
* Function that counts number of pieces on the board

## Setup

### Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.animation as ani
import scipy.optimize as opt
import scipy.stats as sts
import seaborn as sns
%cd ~/Google Drive/MNK/Experiments/0 Two Player Free Play/

### Set Directories

In [ ]:
data_source = 'Data & Analysis/Clean/'
figure_out = 'Data & Analysis/Figures/weiji_talk/'

### Helper Functions

In [ ]:
def detect_error(row):
    is_error = np.nan
    if row.color == 0:  
        is_error = 1 if row.gtv < row.posqual else 0
    else:  
        is_error = 1 if row.gtv > row.posqual else 0  
    return is_error

def array_from_string_rep(stringrep):
    return np.array(list(stringrep)).astype(int)

def count_pieces(stringrep):
    return array_from_string_rep(stringrep).sum()

### Load Data

#### Experimental Data

In [ ]:
data = pd.read_csv(data_source + 'all_data_with_initials0.csv')
data = data.loc[:, data.columns.values[1:]]

data.loc[:, 'num_pieces'] = data.loc[:, 'bp'].map(count_pieces) + data.loc[:, 'wp'].map(count_pieces)

#### Parameters and Fits

In [ ]:
logliks = pd.read_csv(data_source + 'loglik_all.csv')
params_hk = pd.read_csv(data_source + 'params_HK.txt', sep='\t', names=range(15))
bfs_names = ['K0', 'Gamma', 'Lapse Rate', 'Delta', 
             'Vertical Scale', 'Diagonal Scale', 'Opponent Scale', 
             'Feature 0', 'Feature 1', 'Feature 2', 'Feature 3', 
             'Feature 4', 'Feature 5', 'Feature 6', 'Feature 7']
params_bfs = pd.read_csv(data_source + 'params_bfs.txt', sep='\t', names=bfs_names)
logliks = logliks.loc[:37, :]
logliks.loc[:, 'ID'] = logliks.loc[:, 'Initials'].map(dict(zip(data.inits.unique(), np.arange(len(data.inits.unique())))))
meltnames = ['Depth-first', 'Best-first', 'Optrand', 'Notree', 'Myopic', 'Chance']
tidy_ll = pd.melt(logliks, id_vars=['ID'], value_vars=meltnames, 
                  var_name='Model', value_name='ll')

#### Objective values

In [ ]:
objlists = ['objective_values_gen2.txt', 'objective_values_hvh.txt', 'objective_values_eye.txt']
obj = pd.concat([pd.read_csv(data_source + o, sep='\t', 
                            names=['bp', 'wp'] + list(range(36))) 
                for o in objlists]).reset_index(drop=True)

In [ ]:
data.loc[:, 'gtv'] = np.nan
data.loc[:, 'posqual'] = np.nan
data.loc[:, 'error'] = np.nan

for i in data.index.values:
    
    r = int(data.loc[i, 'response'])
    bp = data.loc[i, 'bp']
    wp = data.loc[i, 'wp']
    c = data.loc[i, 'color']
    
    match = obj.loc[(obj.bp == bp) & (obj.wp == wp), :]
    
    if len(match) >= 1:
        gtv = match.loc[:, 35-r].values[0]
        pqual = np.nanmax(match.loc[:, list(range(35))].values[0])
        
        data.loc[i, 'gtv'] = gtv
        data.loc[i, 'posqual'] = pqual if c == 0 else -pqual
        
    data.loc[i, 'error'] = detect_error(data.loc[i, :])

data.loc[0, 'gtv'] = data.loc[0, 'gtv'].astype(float)

### Plotting

In [ ]:
sns.set_palette('deep')
sns.set_style('white')
sns.set_context('poster')
B, G, R, P = sns.color_palette('deep', 4)
%matplotlib inline

## Boilerplate Stuff

In [ ]:
numgames = np.array([data.loc[data.subject==s, 'gi'].max()
        for s in data.subject.unique()
    ])
f, axes = plt.subplots(1, 1, figsize=(10, 5), squeeze=False)
sns.distplot(numgames, ax=axes[0, 0], color='black', kde=False, norm_hist=False, bins=20)
axes[0, 0].set_xlabel('# Games')
axes[0, 0].set_yticklabels([])
axes[0, 0].set_ylabel('Subject')
axes[0, 0].text(x=30, y=7, s='mean    = ' + str(numgames.mean())[:4], fontsize=18)
axes[0, 0].text(x=30, y=6.3, s='median = ' + str(np.median(numgames))[:4], fontsize=18)
sns.despine()
f.savefig(figure_out + 'games_per_subject.png', format='png')

### Moves per game

In [ ]:
nummoves = np.array([data.loc[(data.subject==s)&(data.gi==g), 'mi'].max() 
            for s in data.subject.unique() 
            for g in data.loc[data.subject == s, 'gi'].unique()])

f, axes = plt.subplots(1, 1, figsize=(10, 5), squeeze=False)
sns.distplot(nummoves[np.where(nummoves >= 7)[0]], 
             ax=axes[0, 0], color='black', kde=False, norm_hist=False,
             bins=np.linspace(7, 36, 30))
axes[0, 0].set_xlabel('# Moves')
axes[0, 0].set_yticklabels([])
axes[0, 0].set_ylabel('Games')
axes[0, 0].text(x=21, y=40, s='mean    = ' + str(nummoves.mean())[:4], fontsize=18)
axes[0, 0].text(x=21, y=35, s='median = ' + str(np.median(nummoves))[:4], fontsize=18)
axes[0, 0].set_xlim([7, 36])
sns.despine()
f.savefig(figure_out + 'moves_per_game.png', format='png')

## Model Statistics

### Loglikelihood Comparisons

In [ ]:
g = sns.pairplot(logliks, vars=['Depth-first', 'Best-first', 'Optrand', 'Notree', 'Myopic'],
                palette=sns.color_palette(['#000000']), hue='ID')
line = {'xdata':[0, 3.2], 'ydata':[0, 3.2]}
g.map_offdiag(plt.plot, color='black', alpha=.1, **line)
g.set(xlim=[0, 3.2], ylim=[0, 3.2])

tops = [1, 2, 3, 4, 7, 8, 9, 13, 14, 19]
bottoms = [5, 10, 11, 15, 16, 17, 20, 21, 22, 23, 24]
middles = [0, 6, 12, 18, 24]

for i in tops:
    g.fig.axes[i].set_visible(False)

g._legend.set_visible(False);
g.fig.savefig(figure_out + 'model_crossperformance_with_hists.png', format='png')


### Parameter Histograms

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(15, 20), squeeze=False)

for i in range(15)[1:]:
    axi = i - 1
    curax = axes[axi // 3, axi%3]
    vals = params_bfs.loc[:, bfs_names[i]]
    sns.distplot(vals, color='black', ax=curax, 
                 bins=np.linspace(vals.min(), vals.max(), 21), kde=False)
    curax.set_xlabel(bfs_names[i])
sns.despine()
fig.tight_layout()
fig.savefig(figure_out + "BFS_param_hists.png", format='png')

## Behavioral Statistics

### Response Times

#### Histograms by subject

In [ ]:
def time_hist(data, ax, color='black'):
    rts = data.loc[:, 'rt']
    nrts = rts/rts.sum()
    rt_median = np.median(rts)
    sns.distplot(rts, ax=ax, bins=range(30), color=color, norm_hist=True, kde=False)
#     sns.kdeplot(rts, ax=ax, color=color, shade=True, legend=False, **{'label':''})
    ax.plot([rt_median, rt_median], [0, 1], color=color, 
            linewidth=2, alpha=.7, label="median = " + str(rt_median))
    ax.set_xlim([0, 30])
    ax.set_ylim([0, .4])
    ax.legend()
    return ax
    
f, axes = plt.subplots(10, 2, figsize=(15,50), sharex=True, sharey=True, squeeze=False)

for s in range(19):
    curax = axes[s//2, s%2]
    _s = 2*s
    s1 = data.loc[data.subject==_s, :]
    s2 = data.loc[data.subject==(_s+1), :]
    time_hist(s1, curax)
    time_hist(s2, curax, color=R)
    curax.set_xlabel(str(_s) + ' - ' + str(_s+1))
    curax.set_yticklabels([])

axes[9,1].set_visible(False)
sns.despine()   
f.text(x=.5, y=.91, s='Response Time')
f.savefig(figure_out + "response_hists.png", format='png');

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(10,10), squeeze=False)

x = np.array([np.median(data.loc[data.subject == i, 'rt']) 
              for i in data.loc[data.subject%2 == 1, 'subject'].unique()])
y = np.array([np.median(data.loc[data.subject == i, 'rt']) 
              for i in data.loc[data.subject%2 == 0, 'subject'].unique()])

m, b, r, p, err = sts.linregress(x,y)
xl = np.linspace(0, 30, 11)
yl = m*xl + b

ax = axes[0, 0]
ax.scatter(x, y, c='black')
ax.set_xlim([0,30])
ax.set_xlabel("First Player Median Response")
ax.set_ylim([0, 30])
ax.set_ylabel("Second Player Median Response")
ax.set_title("Median Response Correlation In Opponent Pairs")
f.text(x=.8, y=.8, s="r = " + str(r)[:4], size=18)
f.text(x=.8, y=.77, s="p  = " + str(p)[:6], size=18)
ax.plot(xl, yl, color='black')
sns.despine()
f.savefig(figure_out + "response_correlation.png", format="png")

## Objective Errors

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(20,6), squeeze=False)

sns.kdeplot(data.loc[data.error==0, 'rt'], color='black', ax=axes[0, 0], shade=True, **{'label':'No error'})
sns.kdeplot(data.loc[data.error==1, 'rt'], color=P, ax=axes[0, 0], shade=True, **{'label':'Error'})
axes[0,0].plot([np.median(data.loc[data.error==0, 'rt']), np.median(data.loc[data.error==0, 'rt'])],
              [0, 100], color='black')
axes[0,0].plot([np.median(data.loc[data.error==1, 'rt']), np.median(data.loc[data.error==1, 'rt'])],
              [0, 100], color=P)
axes[0,0].set_xlim([0, 30])
axes[0,0].set_xlabel('Response Time')
axes[0,0].set_ylim([0, .15])
axes[0,0].set_yticklabels([])


sns.barplot('error', 'rt', data=data, ax=axes[0,1], color='grey')
t, p = sts.ttest_ind(data.loc[data.error==0, 'rt'].values, data.loc[data.error==1, 'rt'].values)
axes[0,1].text(x=-.4, y=8.75, s="p = " + str(p), size=18)
axes[0,1].set_xlabel('Error')

sns.despine()
f.savefig(figure_out + 'group_diff_error.png', format='png')


In [ ]:
f, axes = plt.subplots(nrows=9, ncols=4, figsize=(20, 35), squeeze=False, sharex=True, sharey=True)

for i in range(36):
    crax = axes[i//4, i%4]
    sns.distplot(data.loc[data.mi==i, 'rt'], ax=crax)
    crax.set_xlim([0,30])
    crax.set_ylim([0, .5])
    crax.set_xlabel('Num_Pieces = ' + str(i))
    
sns.despine()

In [ ]:
f, axes = plt.subplots(1,1,figsize=(10,10),squeeze=False)

sns.barplot('mi', 'rt', data=data, estimator=np.median, ax=axes[0,0], ci=95)
sns.despine()

In [ ]:
meds = np.array([np.median(data.loc[data.mi==i, 'rt']) for i in range(36)])
mens = np.array([np.mean(data.loc[data.mi==i, 'rt']) for i in range(36)])
sds = np.zeros(36)
sds2 = np.zeros(36)
for i in range(36):
    d = data.loc[data.mi==i, 'rt']
    sds[i] = np.mean(np.abs(d - meds[i]))
    sds2[i] = np.mean(np.abs(d-mens[i]))

plt.plot(np.arange(36), meds)
plt.plot(np.arange(36), sds)
plt.plot(np.arange(36), mens)
plt.plot(np.arange(36), sds2)
sns.despine()

In [ ]:
# f, axes = plt.subplots(nrows=38, ncols=2, figsize=(10,80), 
#                        squeeze=False, sharex=True, sharey=True)

# for i in data.subject.unique():
#     curax = axes[i, 0]
#     sns.distplot(data.loc[(data.subject==i)&(data.error==0), 'rt'], norm_hist=True, ax=curax, 
#                  color='black', kde=False, bins=list(range(30)))
#     curax = axes[i, 1]
#     sns.distplot(data.loc[(data.subject==i)&(data.error==1), 'rt'], norm_hist=True, ax=curax, 
#                  color='black', kde=False, bins=list(range(30)))
    
#     # collapse across subject, kde
#     # mean + std er for two types: rt by error or error for each subject, with error bar
    
# sns.despine()

In [ ]:
errs = pd.DataFrame(data=np.array([data.subject.unique(), 
                       [data.loc[data.subject == s, 'error'].sum() / len(data.loc[data.subject == s, 'error'])
                       for s in data.subject.unique()]]).T, columns=['subject', 'error_rate'])

logliks.loc[:, 'error_rate'] = errs.loc[:, 'error_rate']

f, axes = plt.subplots(1, 2, figsize=(20, 6), squeeze=False)

sns.distplot(errs.error_rate, kde=False, ax=axes[0,0],
             bins=np.linspace(0,1,21), color='black')

axes[0,0].set_xlabel('Error Rate')

sns.regplot('Best-first', 'error_rate', data=logliks, ax=axes[0,1], color='black')

x = logliks.loc[:, 'Best-first']
y = logliks.loc[:, 'error_rate']
m, b, r, p, err = sts.linregress(x,y)

axes[0, 1].text(s='r2 = ' + str(r**2)[:6], x=2.5, y=.7, size=18)
axes[0, 1].text(s='p = ' + str(p)[:6], x=2.5, y=.65, size=18)
axes[0,1].set_xlabel('Best-first NLL')
axes[0,1].set_ylabel('Error Rate')

sns.despine()
f.savefig(figure_out + 'error_rates.png', format='png')


## Movie Time

In [ ]:
test

In [ ]:
s = 4
gi = 3

test = data.loc[data.subject.isin([s, s+1]) & (data.gi == gi), :].copy().reset_index(drop=True)
test.loc[:, 'movietime'] = test.loc[:, 'time'].cumsum() - test.loc[0, 'time']
test.loc[:, 'movietime'] = test.loc[:, 'movietime'] * 1000 / test.loc[:, 'movietime'].max()
test

In [ ]:
def shape_helper(stringrep):
    a = 35 - np.where(array_from_string_rep(stringrep)==1)[0]
    axy = np.zeros([2, a.size])
    axy[0, :] = a % 9 + .5
    axy[1, :] = a // 9 + .5
    return axy
    
def shape_position(row):
    b = shape_helper(row.bp)
    w = shape_helper(row.wp)
    return b, w

def shape_move(row):
    r = 35 - row.response
    return np.array([r % 9 + .5, r // 9 + .5]).T

def plot_position(row):
    msiz = 6000
#     b, w = shape_position(row)
#     plt.scatter(b[0], b[1], c='black', s=msiz)
#     plt.scatter(w[0], w[1], c='white', s=msiz)
    if int(row.color) == 0:
        c = 'black'
    else:
        c = 'white'
    m = shape_move(row)
    ax = plt.gca()
    ax.scatter(m[0], m[1], c=c, s=msiz)
    return ax

def movie_init():
    ax = plt.gca()
    ax.set_xlim([0,9])
    ax.set_ylim([0,4])
    ax.set_aspect(1)
    ax.grid()
    ax.set_xticks(np.arange(10))
    ax.set_yticks(np.arange(5))
    ax.set_axis_bgcolor('grey')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    sns.despine()
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    return ax

nframes=550

def make_frames(i):
    rao = test.loc[test.movietime <= 2 * i, :]
    raow = rao.index.values[-1] if len(rao) > 1 else 0
    raow = test.loc[raow, :]
    if 2 * i > test.loc[0, 'movietime']:
        ax = plot_position(raow)
    else:
        ax = plt.gca()
    return ax

f, axes = plt.subplots(1, 1, figsize=(15, 15), squeeze=False)

movie = ani.FuncAnimation(f, make_frames, init_func=movie_init, frames=nframes, interval=100, blit=False);
movie.save(figure_out + 'test_movie3.mp4', writer='mencoder');